<a href="https://colab.research.google.com/github/fastdatascience/clinical_trial_risk/blob/fixes_nov_2022/train/ctgov/TrainNumArmsClassifier_22_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import spacy
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, auc, roc_curve
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

In [116]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
df_annotations = pd.read_csv("/content/drive/MyDrive/data/all_annotations.csv")


# Get data

In [118]:
stats_stopwords = set("""A&H
Abbott
Actelion
Aesica pharmaceuticals
Air Products
Alcon
ALK-Abello
Allergan
Alpharma
Almus Pharmaceuticals
Altana
Amgen
APS
Assertio
AstraZeneca
Aventis Pharma
Bausch & Lomb
Baxter
Baxter BioScience
Bayer
Becton Dickinson
Beiersdorf
Berk
Biogen
Boehringer Ingelheim
Boots
BPL
Braun
Bristol-Myers Squibb
Celltech
Cephalon
Chemidex
Chugai
Colgate-Palmolive
Coloplast
ConvaTec
Cow & Gate
CP
Crookes
Dentsply
Dista
DuPont
Eisai
Ethicon
Fabre
Ferring
Florizel
Fresenius SE
Galderma
Galpharm Healthcare
Garnier
GE Healthcare
Gilead
GlaxoSmithKline
Grünenthal
Grifols
GSK
GSK Consumer Healthcare
Heinz
Hillcross
Hoechst Marion Roussel
Invicta
Ipsen
ITH Pharma Ltd
IVAX
J&J
J&J Medical
J&J MSD
Janssen-Cilag
King
Kyowa Hakko
Lederie
LEO
LifeScan
Lilly
Lundbeck
Martindale Pharma
Mölnlycke
Mayne
McNeil
Meda
Medac
MediSense
Menarini
Menarini Diagnostics
Merck
Merck Consumer Health
Merck & Co.
Milupa
MSD
Nestlé
Nestlè Clinical
Neutrogena
Novartis
Novartis Consumer Health
Novo Nordisk
Nycomed
Octapharma
Orion
Otsuka
Parke-Davis
Pfizer
Pharmacia
Procter & Gamble
Procter & Gamble Pharm.
Reckitt Benckiser
Rhône-Poulenc
Roche
Roche Consumer Health
Roche Diagnostics
Rosemont
Rybar
Salts
Sandoz
Sankyo
Sanofi-Aventis
Sanofi Oasteur
Sanofi-Synthélabo
Schering-Plough
Schwarz
Searle
Serono
Servier
Shire
SHS
Sigma
SMA Nutrition
SNBTS
Solvay
Squibb
SSL
Stiefel
Takeda
Taro
Teva
Thornton & Ross
Trinity-Chiesi
Tyco
Univar
Valeant
Viatris
W-L
Warner Lambert
Wockhardt
Wyeth
Yamanouchi
title
study
protocol
participant
participants
irb
africa
trial
patient
patients
treatment
short
trial
gov
sponsor
university
delivery
co
usa
phone
edu
principal
investigators
laboratory
department
us
email
institute
centre
com
confidentiality
confidential
information
must
investigator
team
relevant
ethics
regulatory
contents
research
non
summary
background
rationale
primary
sites
population
interventions
pubmed
January
February
March
April
May
June
July
August
September
October
November
December
Jan
Feb
Mar
Apr
May
Jun
Jul
Aug
Sep
Oct
Nov
Dec
Monday
Tuesday
Wednesday
Thursday
Friday
Saturday
Sunday
Mon
Tue
Wed
Thu
Fri
Sat
Sun
recommendations
length
experienced
making
contain
girls
youth
sms
mobile
economic
uptake
vac
rotavirus
vivax
azithromycin
parasite
stool
placental
village
malaria
ya
plasmodium
png
toc
path
iron
measles
anaemia
vitamin
ultrasound
chloroquine
aerosol
supplementation
gestation
dp
cq
transfusion
gestational
skip
pr
iga
ia
vials
relapse
br
pp
growth
tropical
parents
vi
mass
triple
fetal
anemia
endemic
postnatal
diarrhea
feeding
antenatal
water
food
swabs
pcr
nurse
specify
titers
cmc
workers
hvtn
taf
chc
impaact
rpv
cab
mab
hptn
ral
itraconazole
dtg
linkage
amphotericin
ip
alvac
nichd
ftc
wrair
mva
ts
hiv
antepartum
tdf
aidsvax
vrc
efv
ring
crs
infusion
env
daids
subtype
hbv
psrt
cq
rv
viral
vaginal
arv
bivalent
arvs
zdv
sc
meningitis
sti
mcg
virologic
suppression
antiretroviral
combinations
ucsf
neutralizing
counseling
bid
niaid
tenofovir
abc
aeras
dar
qft
bedaquiline
stream
bdq
mtb
bcg
abidjan
sputum
mdr
tb
anrs
alliance
qtcf
moxifloxacin
xpert
att
cld
inh
ppd
igra
ics
tuberculosis
ltbi
iris
pyrazinamide
biologicals
prednisone
mrc
qt
rct
pa
atp
genexpert
proprietary
hair
dst
imp
rif
quantiferon
ifn
si
idmc
ctu
smc
reproduction
injection
shells
multidrug
isoniazid
pulmonary
corticosteroids
focused
mycobacterium
unfavourable
reserved
ds
tubes
lp
vaccinated
cytokine
copyright
qtc
suspicion
allowable
vaccination
ipt
favourable
stimulation
coordination
rifampicin
ray
tnf
pbmcs
cultures
immunology
csf
uln
revaccination
profiling
iec
ap
baby
al
tel
org
community
mailto
md
doi
pk
fax
ac
medicine
communities
uk
pmid
srl
survey
epub
appendix
plasma
prevention
srvge
rawres
rvge
pt
wise
dehydration
serotype
fifth
cut
reactogenicity
whose
stools
vomiting
immunity
immunogenicity
expressing
characteristics
exact
circumference
preferred
intussusception
postpartum
episodes
added
missing
across
yes
prenatal
difference
symptom
markers
end
gastroenteritis
third
maternal
age
second
enrolment
objective
code
month
last
overall
included
populations
excluded
case
blinded
year
term
severity
done
start
group
lead
subjects
definition
subject
definitions
unsolicited
cases
cohort
cohorts
least
days
post
month
months
level
levels
among
per
size
sizes
figure
figures
comparisons
date
dates
birth
births
hospitalization
hospitalizations
hospitalisation
hospitalisations
hospital
hospitals
woman
women
time
times
considered
weight
weights
never
number
dated
listing
breastfeeding
introduction
arm
arms
data
using
use
score
source
use
used
uses
protocol
protocols
subject
subjects
trial
trials
doctor
doctors
eg
get
getting
got
gotten
rx
dapivirine""".split("\n"))

In [119]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [120]:
import pycountry


In [121]:
tok = RegexpTokenizer(r'[a-zA-Z]+')

stops = set(stopwords.words('english')).union(set(stopwords.words('french'))).union(stats_stopwords)

for c in pycountry.countries:
    for t in tok.tokenize(c.name.lower()):
        if t not in ("monte", "carlo"):
            stops.add(t)

In [122]:
with open("/content/drive/MyDrive/data/protocols.pkl.gz", "rb") as f:
    file_to_pages = pkl.load(f)

In [123]:
import operator
texts = []

for i in range(len(df_annotations)):
    file_name = df_annotations.file.iloc[i]
    
    pages= file_to_pages[file_name]

    texts.append(" ".join(pages))
df_annotations["text"] = texts

In [124]:
 del file_to_pages

In [125]:
df_annotations = df_annotations[~df_annotations.num_arms.isna()]

In [126]:
def get_num_arms_clean(num):
    if pd.isna(num):
        return None
    if num >= 3:
        return "3+"
    return str(int(num))
df_annotations["num_arms_clean"] = df_annotations["num_arms"].apply(get_num_arms_clean)

In [127]:
df_annotations["num_arms_clean"].value_counts()

2     5228
3+    3277
1     3086
Name: num_arms_clean, dtype: int64

In [ ]:
labels = list(sorted(set(df_annotations["num_arms_clean"])))

In [128]:
df_train = df_annotations[df_annotations.train_val == "train"]
df_val = df_annotations[df_annotations.train_val == "val"]

In [212]:

# Specially engineered regex to include 95%, 95%ci, etc
vectoriser = CountVectorizer(lowercase=True, stop_words = stops, min_df=5,
                            token_pattern=r'\w+', ngram_range=[1,2], max_features=1000)

transformer = TfidfTransformer()

nb = MultinomialNB()
model = make_pipeline(vectoriser, transformer, nb)

In [213]:
df_train_small = df_train.sample(1000)

In [214]:
model.fit(df_train_small.text, df_train_small.num_arms_clean)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abbott', 'abello', 'actelion', 'aesica', 'air', 'alcon', 'alk', 'allergan', 'almus', 'alpharma', 'altana', 'amgen', 'apr', 'april', 'aps', 'assertio', 'astrazeneca', 'aug', 'august', 'aventis', 'bausch', 'baxter', 'bayer', 'becton', 'beiersdorf', 'benckiser', 'berk', 'biogen', 'bioscience', 'boehringer', 'boots', 'bpl', 'braun', 'bristol', 'celltech', 'cephalon', 'chemidex', 'chiesi', 'chugai', 'cilag', 'clinical', 'colgate', 'coloplast', 'consumer', 'convatec', 'cow', 'cp', 'crookes', 'davis', 'dec', 'december', 'dentsply', 'diagnostics', 'dickinson', 'dista', 'dupont', 'eisai', 'ethicon', 'fabre', 'feb', 'february', 'ferring', 'florizel', 'fresenius', 'fri', 'friday', 'galderma', 'galpharm', 'gamble', 'garnier', 'gate', 'ge', 'gilead', 'glaxosmithkline', 'grifols', 'grünenthal', 'gsk', 'h'

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=1000, min_df=5,
                                 ngram_range=[1, 2],
                                 stop_words={'A&H', 'ALK-Abello', 'APS',
                                             'Abbott', 'Actelion',
                                             'Aesica pharmaceuticals',
                                             'Air Products', 'Alcon',
                                             'Allergan',
                                             'Almus Pharmaceuticals',
                                             'Alpharma', 'Altana', 'Amgen',
                                             'Apr', 'April', 'Assertio',
                                             'AstraZeneca', 'Aug', 'August',
                                             'Aventis Pharma', 'BPL',
                                             'Bausch & Lomb', 'Baxter',
                                             'Baxter BioScience', 'Bayer'

In [207]:
df_val["y_pred"] = model.predict(df_val.text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [208]:
df_val["y_pred"]

12       2
49       2
50       2
51       2
52       2
        ..
11902    2
11903    2
11904    2
11906    2
11907    2
Name: y_pred, Length: 1056, dtype: object

In [209]:
y_pred_proba = model.predict_proba(df_val.text)

KeyboardInterrupt: ignored

In [ ]:
y_pred_proba

In [ ]:

ground_truths = df_val.num_arms_clean
pred = df_val.y_pred

In [ ]:
pred

In [ ]:
acc = accuracy_score(ground_truths, pred)
print (f"accuracy {acc}")

In [ ]:
pred

In [ ]:
ConfusionMatrixDisplay.from_predictions(ground_truths, pred)
plt.xticks(rotation=90)
;

In [ ]:
all_aucs = []
from sklearn.metrics import roc_auc_score
for i, label in enumerate(list(nb.classes_)):
  roc_auc = roc_auc_score(ground_truths==label, y_pred_proba[:,i])
  print (f"ROC AUC {roc_auc}")
  all_aucs.append(roc_auc)
print (np.mean(all_aucs))

In [ ]:
plt.figure()
lw = 2
    
for i, label in enumerate(list(nb.classes_)):
    fpr, tpr, _ = roc_curve(ground_truths==label, y_pred_proba[:,i])
    roc_auc = auc(fpr, tpr)
    plt.plot(
        fpr,
        tpr,
        # color="darkorange",
        lw=lw,
        label=f"ROC curve arms = {label} (area = {roc_auc:0.2f})"
    )
    plt.plot([0, 1], [0, 1], color="gray", alpha=0.5, lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()

In [210]:
vectoriser.vocabulary_

{'amendment': 47963,
 '2': 11083,
 '23': 17426,
 'dec': 97854,
 '2016': 15663,
 'page': 208327,
 '106': 5133,
 'changes': 74749,
 'version': 294921,
 'approval': 53334,
 '3': 19346,
 '0': 0,
 'integrated': 166253,
 '1': 1621,
 '15': 8288,
 'jul': 170958,
 '2015': 15497,
 'original': 205770,
 '20': 13335,
 'aug': 59437,
 '2014': 15319,
 'amended': 47917,
 'reissued': 241724,
 '11': 5365,
 'signature': 263158,
 'undersigned': 290341,
 'confirms': 86613,
 'agrees': 45321,
 'conduct': 85608,
 'conditions': 85326,
 'described': 100467,
 'comply': 84060,
 'international': 167128,
 'conference': 86202,
 'harmonization': 148736,
 'good': 145869,
 'clinical': 78023,
 'practice': 221437,
 'ich': 155114,
 'gcp': 143566,
 'applicable': 52033,
 'requirements': 246808,
 '___________________________________': 35342,
 '______________________': 35326,
 'name': 195156,
 'please': 218455,
 'print': 224334,
 'location': 177701,
 '4': 23145,
 'synopsis': 276919,
 'multicenter': 194369,
 'randomized': 23366

In [215]:
fake_document = " ".join(vectoriser.vocabulary_)
vectorised_document = vectoriser.transform([fake_document])
transformed_document = transformer.transform(vectorised_document)
probas = np.zeros((transformed_document.shape[1]))

for prediction_idx in [0, 1, 2]:
      print(f"Strongest predictors for class {nb.classes_[prediction_idx]}\n")
      for i in range(transformed_document.shape[1]):
          zeros = np.zeros(transformed_document.shape)
          zeros[0, i] = transformed_document[0, i]
          proba = nb.predict_log_proba(zeros)
          probas[i] = proba[0, prediction_idx]

      for ctr, j in enumerate(np.argsort(-probas)):
          for w, i in vectoriser.vocabulary_.items():
              if i == j:
                  print(f"{ctr}\t{w}")
          if ctr> 50:
            break

Strongest predictors for class 1

0	grade
1	phase ii
2	marrow
3	pharmaceuticals inc
4	pf
5	h h
6	pet
7	tumors
8	ct
9	metastatic
10	bi
11	bone
12	progression
13	transplant
14	measurable
15	bone marrow
16	cycle
17	escalation
18	tumor
19	nci
20	oncology
21	ati
22	lesions
23	pharmaceuticals
24	dlt
25	cycles
26	radiation
27	ctcae
28	advanced
29	imaging
30	toxicities
31	cycle 1
32	lesion
33	ii
34	w h
35	cr
36	disease progression
37	grade 1
38	grade 3
39	cancer
40	grade 2
41	08
42	chemotherapy
43	open label
44	toxicity
45	survival
46	m2
47	pd
48	mri
49	expression
50	dose escalation
51	biopsy
Strongest predictors for class 2

0	e
1	e e
2	_
3	h e
4	_ _
5	double blind
6	insulin
7	h
8	blind
9	ati
10	di
11	2020
12	6
13	double
14	version
15	randomization
16	visit
17	placebo
18	p
19	10
20	e nt
21	care
22	alcohol
23	version 1
24	consent
25	nt
26	randomized
27	surgery
28	4
29	diabetes
30	5
31	medical
32	7
33	w e
34	g
35	8
36	cti
37	respiratory
38	form
39	pi
40	standard
41	test
42	version 5
43	v
44	ot


In [202]:
import bz2
with bz2.open("arms_classifier_document_level.pkl.bz2", "wb") as f:
    pkl.dump(model, f)

In [187]:
1

1